In [1]:
import os
import csv
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('normalized_medical_expensess.db')
cursor = conn.cursor()

# Create tables
cursor.execute('''CREATE TABLE IF NOT EXISTS numerical_data (
                    id INTEGER PRIMARY KEY,
                    age INTEGER,
                    bmi REAL,
                    children INTEGER,
                    charges REAL
                )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS categorical_data (
                    id INTEGER PRIMARY KEY,
                    sex TEXT,
                    smoker TEXT,
                    region TEXT
                )''')

# Read data from CSV and insert into tables
with open('medical insurance expensess-Copy1.csv', newline='') as file:
    reader = csv.DictReader(file)
    for row in reader:
        # Insert into numerical_data table
        numerical_data = (row['age'], row['bmi'], row['children'], row['charges'])
        cursor.execute('''INSERT INTO numerical_data (age, bmi, children, charges) VALUES (?, ?, ?, ?)''', numerical_data)
        numerical_id = cursor.lastrowid
        
        # Insert into categorical_data table
        categorical_data = (row['sex'], row['smoker'], row['region'])
        cursor.execute('''INSERT INTO categorical_data (sex, smoker, region) VALUES (?, ?, ?)''', categorical_data)
        categorical_id = cursor.lastrowid
        
        # Update numerical_data table with categorical_id
        cursor.execute('''UPDATE numerical_data SET id = ? WHERE rowid = ?''', (categorical_id, numerical_id))

# Commit changes and close connection
conn.commit()
conn.close()

In [2]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('normalized_medical_expensess.db')
cursor = conn.cursor()

# Retrieve and print numerical data
print("Numerical Data:")
cursor.execute('''SELECT * FROM numerical_data''')
numerical_data = cursor.fetchall()
for row in numerical_data:
    print(row)

# Retrieve and print categorical data
print("\nCategorical Data:")
cursor.execute('''SELECT * FROM categorical_data''')
categorical_data = cursor.fetchall()
for row in categorical_data:
    print(row)

# Close connection
conn.close()


Numerical Data:
(1, 19, 27.9, 0, 16884.924)
(2, 18, 33.77, 1, 1725.5523)
(3, 28, 33.0, 3, 4449.462)
(4, 33, 22.705, 0, 21984.47061)
(5, 32, 28.88, 0, 3866.8552)
(6, 31, 25.74, 0, 3756.6216)
(7, 46, 33.44, 1, 8240.5896)
(8, 37, 27.74, 3, 7281.5056)
(9, 37, 29.83, 2, 6406.4107)
(10, 60, 25.84, 0, 28923.13692)
(11, 25, 26.22, 0, 2721.3208)
(12, 62, 26.29, 0, 27808.7251)
(13, 23, 34.4, 0, 1826.843)
(14, 56, 39.82, 0, 11090.7178)
(15, 27, 42.13, 'None', 39611.7577)
(16, 19, 24.6, 1, 1837.237)
(17, 52, 30.78, 1, 10797.3362)
(18, 23, 23.845, 0, 2395.17155)
(19, 56, 40.3, 0, 10602.385)
(20, 30, 'None', 0, 36837.467)
(21, 60, 36.005, 0, 13228.84695)
(22, 30, 32.4, 1, 4149.736)
(23, 18, 34.1, 0, 1137.011)
(24, 34, 31.92, 1, 37701.8768)
(25, 37, 28.025, 2, 6203.90175)
(26, 59, 27.72, 3, 14001.1338)
(27, 63, 23.085, 0, 14451.83515)
(28, 55, 32.775, 2, 12268.63225)
(29, 23, 17.385, 1, 2775.19215)
(30, 31, 36.3, 2, 38711.0)
(31, 22, 35.6, 0, 35585.576)
(32, 18, 26.315, 0, 2198.18985)
(33, 19, 28.6, 

In [3]:
import pandas as pd
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('normalized_medical_expensess.db')

# SQL query with JOIN
query = """ SELECT numerical_data.id,numerical_data.age,numerical_data.bmi,numerical_data.children,
                   categorical_data.sex,categorical_data.region,categorical_data.smoker,numerical_data.charges
            FROM numerical_data
            INNER JOIN categorical_data ON categorical_data.id = numerical_data.id;
        """

# Fetch data into Pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close connection
conn.close()

# Display DataFrame
print(df)


        id age     bmi children     sex     region smoker      charges
0        1  19    27.9        0  female  southwest    yes    16884.924
1        2  18   33.77        1    male  southeast     no    1725.5523
2        3  28    33.0        3    male  southeast     no     4449.462
3        4  33  22.705        0    male  northwest     no  21984.47061
4        5  32   28.88        0    male  northwest     no    3866.8552
...    ...  ..     ...      ...     ...        ...    ...          ...
1333  1334  50   30.97        3    male  northwest     no   10600.5483
1334  1335  18   31.92        0  female  northeast     no    2205.9808
1335  1336  18   36.85        0  female  southeast     no    1629.8335
1336  1337  21    25.8        0  female  southwest     no     2007.945
1337  1338  61   29.07        0  female  northwest    yes   29141.3603

[1338 rows x 8 columns]


In [4]:
df.dtypes

id           int64
age         object
bmi         object
children    object
sex         object
region      object
smoker      object
charges     object
dtype: object

In [6]:
df.describe()

,id
count,1338.000000
mean,669.500000
std,386.391641
min,1.000000
25%,335.250000
50%,669.500000
75%,1003.750000
max,1338.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1338 non-null   int64 
 1   age       1338 non-null   object
 2   bmi       1338 non-null   object
 3   children  1338 non-null   object
 4   sex       1338 non-null   object
 5   region    1338 non-null   object
 6   smoker    1338 non-null   object
 7   charges   1338 non-null   object
dtypes: int64(1), object(7)
memory usage: 83.8+ KB


In [10]:
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
train, test = train_test_split(df, test_size=0.3, stratify=df['sex'], random_state=20240417)

In [11]:
train['sex'].value_counts() / len(train['sex'])
test['sex'].value_counts() / len(test['sex'])

sex
male      0.505342
female    0.494658
Name: count, dtype: float64

sex
male      0.504975
female    0.495025
Name: count, dtype: float64

In [12]:
train.drop('sex',axis=1,inplace=True)
test.drop('sex',axis=1,inplace=True)

In [13]:
train.head()

,id,age,bmi,children,region,smoker,charges
1220,1221,30,21.945,1,northeast,no,4718.20355
397,398,21,31.02,0,southeast,no,16586.49771
148,149,53,37.43,1,northwest,no,10959.6947
113,114,21,35.72,0,northwest,no,2404.7338
1020,1021,51,37.0,0,southwest,no,8798.593
